In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from math import inf
import pandas as pd

In [18]:
def get_url(type, location, transaction, page):
    return f"https://www.fincaraiz.com.co/{type}/{transaction}/{location}?pagina={page}"

In [19]:
def get_driver(disable_gui = True):
    options = Options()
    if disable_gui:
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    return(webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options))

In [20]:
transaction = "venta"
city = "medellin"
department = "antioquia"
location = f"{city}/{department}"
type = "apartaestudios-apartamentos"
houses = []

In [21]:
def get_information(url: str):
    page_driver = get_driver()
    page_driver.get(url)
    attributes_container = page_driver\
        .find_element(By.ID, "general")\
        .find_elements(By.XPATH, '*')[0]\
        .find_elements(By.XPATH, '*')[1]\
        .find_elements(By.XPATH, '*')[0]
    attributes = attributes_container.find_elements(By.CSS_SELECTOR, "p.jss65.jss74")
    house = {}
    for index in range(len(attributes)):
        if index % 2 != 0:
            continue
        key = attributes[index].text
        value = attributes[index + 1].text
        try:
            if "m²" in value or "&nbsp;" in value or "*":
                value = value.replace("m²", "").replace("&nbsp;", "").replace("*", "").strip()
            house[key] = value
        except Exception as e:
            print(f"Error al obtener el atributo '{key}'")
            print(e)
            pass
    try:
        price = page_driver.find_element(By.TAG_NAME, "aside").find_element(By.XPATH, "//p[text()='Precio total (COP)']/following-sibling::p")
        house["Precio total (COP)"] = price.get_attribute("innerHTML").replace("&nbsp;", "").strip()
    except Exception:
        pass
    houses.append(house)
    page_driver.close()

In [22]:
main_driver = get_driver()
max_per_page = inf
try:
    for page in range(1, 30):
        main_url = get_url(type, location, transaction, page)
        main_driver.get(main_url)
        articles = main_driver.find_elements(By.TAG_NAME, "article")
        total_articles = len(articles)
        print(f"Página {page}")
        print(f"Esta página contiene {total_articles} elementos")
        houses_iterated = 0
        for index, article in enumerate(articles):
            if houses_iterated == max_per_page:
                break
            print(f"Obteniendo información del inmueble {index + 1} de {total_articles}")
            try:
                url = article.find_element(By.TAG_NAME, "a").get_property("href")
                print(url)
            except Exception:
                print("Error al obtener la URL")
                continue
            get_information(url)
            houses_iterated += 1
finally:
    main_driver.close()


Página 1
Esta página contiene 33 elementos
Obteniendo información del inmueble 1 de 33
https://www.fincaraiz.com.co/proyecto-de-vivienda/bit-plus/los-conquistadores/medellin/7969479
Obteniendo información del inmueble 2 de 33
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/loma-de-los-bernal/medellin/8087163
Obteniendo información del inmueble 3 de 33
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/loma-del-indio/medellin/10178995
Obteniendo información del inmueble 4 de 33
https://www.fincaraiz.com.co/proyecto-de-vivienda/living-97/veinte-de-julio/medellin/7022668
Obteniendo información del inmueble 5 de 33
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/laureles/medellin/10048183
Obteniendo información del inmueble 6 de 33
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/desconocido/medellin/7831759
Obteniendo información del inmueble 7 de 33
https://www.fincaraiz.com.co/inmueble/apartamento-en-venta/la-milagrosa/medellin/10176438
Obteniendo

KeyboardInterrupt: 

In [23]:
len(houses)

39

In [24]:
houses

[{'Estrato': '4', 'Estado': 'Sobre planos'},
 {'Habitaciones': '3',
  'Baños': '2',
  'Parqueaderos': '1',
  'Área construída': '84',
  'Área privada': '84',
  'Estrato': '5',
  'Antigüedad': '1 a 8 años',
  'Piso N°': '6',
  'Administración': '$ 544.000 COP',
  'Precio m²': '$ 6.369.047,62',
  'Precio total (COP)': '$535.000.000'},
 {'Habitaciones': '2',
  'Baños': '1',
  'Área construída': '37',
  'Área privada': '0',
  'Estrato': '3',
  'Antigüedad': '1 a 8 años',
  'Administración': '$ 100.000 COP',
  'Precio m²': '$ 4.864.864,86',
  'Precio total (COP)': '$180.000.000'},
 {'Estrato': '3', 'Estado': 'En construcción'},
 {'Habitaciones': '3',
  'Baños': '2',
  'Área construída': '129',
  'Área privada': '129',
  'Estrato': '4',
  'Antigüedad': '9 a 15 años',
  'Piso N°': '2',
  'Administración': 'No definida',
  'Precio m²': '$ 3.488.372,09',
  'Precio total (COP)': '$450.000.000'},
 {'Habitaciones': '6',
  'Baños': '7',
  'Área construída': '1.200',
  'Área privada': '0',
  'Estrat

In [25]:
df = pd.DataFrame.from_dict(houses)
df.to_csv(f"house_prices_colombia_{city}.csv")

In [ ]:
df